# MCMC Sampling

## Overview

Stan's MCMC sampler implements the Hamiltonian Monte Carlo (HMC) algorithm and its adaptive variant
the no-U-turn sampler (NUTS).
It creates a set of draws from the posterior distribution of the model conditioned on the data,
allowing for exact Bayesian inference of the model parameters.
Each draw consists of the values for all parameter, transformed parameter, and
generated quantities variables, reported on the constrained scale.

The [CmdStanModel sample](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanModel.sample) method
wraps the CmdStan [sample](https://mc-stan.org/docs/cmdstan-guide/mcmc-config.html) method.
Underlyingly, the CmdStan outputs are a set of per-chain Stan CSV files.
In addition to the resulting sample, reported as one row per draw,
the Stan CSV files encode information about the inference engine configuration
and the sampler state.
The NUTS-HMC adaptive sampler algorithm also outputs the per-chain
HMC tuning parameters `step_size` and `metric`.

The `sample` method returns a [CmdStanMCMC](https://mc-stan.org/cmdstanpy/api.html#cmdstanmcmc) object,
which provides access to the disparate information from the Stan CSV files.
Accessor functions allow the user
to access the sample in whatever data format is needed for further analysis,
either as tabular data (i.e., in terms of the per-chain CSV file rows and columns),
or as structured objects which correspond to the variables in the Stan model
and the individual diagnostics produced by the inference method.


- The [stan_variable](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.stan_variable) and
[stan_variables](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.stan_variables) methods 
return a Python [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray)
containing all draws from the sample where the structure of each draw corresponds to the structure of the
Stan variable.

- The [draws](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.draws) method returns the sample as either a 2-D or 3-D numpy.ndarray.

- The [draws_pd](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.draws_pd) method returns the entire sample or selected variables as a pandas.DataFrame.

- The [draws_xr](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.draws_xr) method returns a structured Xarray dataset over the Stan model variables.

- The [method_variables](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.method_variables) returns a
Python dict over all sampler method variables.


In addition, the `CmdStanMCMC` object has accessor methods for

- The per-chain HMC tuning parameters `step_size` and `metric` 

- The CmdStan run configuration and console outputs

- The mapping between the Stan model variables and the corresponding CSV file columns.

### Notebook prerequisites


CmdStanPy displays progress bars during sampling via use of package [tqdm](https://github.com/tqdm/tqdm).
In order for these to display properly in a Jupyter notebook, you must have the 
[ipywidgets](https://ipywidgets.readthedocs.io/en/latest/index.html) package installed,
and depending on your version of Jupyter or JupyterLab, you must enable it via command:

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

For more information, see the the
[installation instructions](https://ipywidgets.readthedocs.io/en/latest/user_install.html#), 
also [this tqdm GitHub issue](https://github.com/tqdm/tqdm/issues/394#issuecomment-384743637).


    

## Fitting the model and data

In this example we use the CmdStan example model
[bernoulli.stan](https://github.com/stan-dev/cmdstanpy/blob/master/test/data/bernoulli.stan)
and data file
[bernoulli.data.json](https://github.com/stan-dev/cmdstanpy/blob/master/test/data/bernoulli.data.json>).

We instantiate a `CmdStanModel` from the Stan program file

In [ ]:
import os
from cmdstanpy import CmdStanModel

# instantiate, compile bernoulli model
model = CmdStanModel(stan_file='bernoulli.stan')

By default, the model is compiled during instantiation.  The compiled executable is created in the same directory as the program file.  If the directory already contains an executable file with a newer timestamp, the model is not recompiled.

We run the sampler on the data using all default settings:  4 chains, each of which runs 1000 warmup and sampling iterations.

In [ ]:
# run CmdStan's sample method, returns object `CmdStanMCMC`
fit = model.sample(data='bernoulli.data.json')

The `CmdStanMCMC` object records the command, the return code, and the paths to the sampler output csv and console files.  The sample is lazily instantiated on first access of either the draws or the HMC tuning parameters, i.e., the step size and metric.

The string representation of this object displays the CmdStan commands and the location of the output files.
Output filenames are composed of the model name, a timestamp in the form YYYYMMDDhhmmss and the chain id, plus the corresponding filetype suffix, either '.csv' for the CmdStan output or '.txt' for the console messages, e.g. bernoulli-20220617170100_1.csv.

In [ ]:
fit

In [ ]:
print(f'draws as array:  {fit.draws().shape}')
print(f'draws as structured object:\n\t{fit.stan_variables().keys()}')
print(f'sampler diagnostics:\n\t{fit.method_variables().keys()}')

### Sampler Progress

Your model make take a long time to fit.  The `sample` method provides two arguments:
    
- visual progress bar:  `show_progress=True`
- stream CmdStan output to the console - `show_console=True`

By default, CmdStanPy displays a progress bar during sampling, as seen above.
Since the progress bars are only visible while the sampler is running and the bernoulli example model takes no time at all to fit, we run this model for 200K iterations, in order to see the progress bars in action.

In [ ]:
fit = model.sample(data='bernoulli.data.json', iter_warmup=100000, iter_sampling=100000, show_progress=True)


To see the CmdStan console outputs instead of progress bars, specify ``show_console=True``
This will stream all CmdStan messages to the terminal while the sampler is running.
This option will allow you to debug a Stan program using the Stan language `print` statement.

In [ ]:
fit = model.sample(data='bernoulli.data.json', chains=2, parallel_chains=1, show_console=True)



## Checking the fit

The first question to ask of the `CmdStanMCMC` object is:  _is this a valid sample from the posterior?_

It is important to check whether or not the sampler was able to fit the model given the data.  Often, this is not possible, for any number of reasons.
To appreciate the sampler diagnostics, we use a hierarchical model which, given a small amount of data, encounters difficulty: the centered parameterization of the 
"8-schools" model (Rubin, 1981).
The "8-schools" model is a simple hierarchical model, first developed on a dataset taken from
an experiment was conducted in 8 schools, with only treatment effects and their standard errors reported.

The Stan model and the original dataset are in files `eight_schools.stan` and `eight_schools.data.json`.

**eight_schools.stan**

In [ ]:
with open('eight_schools.stan', 'r') as fd:
    print(fd.read())

**eight_schools.data.json**

In [ ]:
with open('eight_schools.data.json', 'r') as fd:
    print(fd.read())

Because there is not much data, the geometry of posterior distribution is highly curved, 
thus the sampler may encounter difficulty in fitting the model.
By specifying the initial seed for the pseudo-random number generator,
we insure that the sampler will have difficulty in fitting this model.
In particular, some post-warmup iterations diverge, resulting in a biased sample.
In addition, some post-warmup iterations hit the maximum allowed treedepth before
the trajectory hits the "U-turn" condition of the NUTS algorithm,
in which case the sampler may fail to properly explore the entire posterior.

These diagnostics are checked for automatically at the end of each run; if problems are detected, a WARNING message is logged.

In [ ]:
eight_schools_model = CmdStanModel(stan_file='eight_schools.stan')
eight_schools_fit = eight_schools_model.sample(data='eight_schools.data.json', seed=55157)

More information on how to address convergence problems can be found at https://mc-stan.org/misc/warnings

The number of post-warmup divergences and iterations which hit the maximum treedepth can be inspected directly via properties `divergences` and `max_treedepths`.

In [ ]:
print(f'divergences:\n{eight_schools_fit.divergences}\niterations at max_treedepth:\n{eight_schools_fit.max_treedepths}')

### Summarizing the sample

The `summary` method reports the R-hat statistic, a measure of how well the sampler chains have converged.

In [ ]:
eight_schools_fit.summary()

### Sampler Diagnostics

The `diagnose()` method provides more information about the sample.

In [ ]:
print(eight_schools_fit.diagnose())

## Accessing the sampler outputs

In [ ]:
fit = model.sample(data='bernoulli.data.json')

### Extracting the draws as structured Stan program variables

Per-variable draws can be accessed as either a numpy.ndarray object
via method `stan_variable` or as an xarray.Dataset object via `draws_xr`.

In [ ]:
print(fit.stan_variable('theta'))

The `stan_variables` method returns a Python `dict` over all Stan variables in the output.

In [ ]:
for k, v in fit.stan_variables().items():
    print(f'name: {k}, shape: {v.shape}')

In [ ]:
print(fit.draws_xr('theta'))

### Extracting the draws in tabular format

The sample can be accessed either as a `numpy` array or a pandas `DataFrame`:

In [ ]:
print(f'sample as ndarray: {fit.draws().shape}\nfirst 2 draws, chain 1:\n{fit.draws()[:2, 0, :]}')

In [ ]:
fit.draws_pd().head()

### Extracting sampler method diagnostics

In [ ]:
for k, v in fit.method_variables().items():
    print(f'name: {k}, shape: {v.shape}')

### Extracting the per-chain HMC tuning parameters

In [ ]:
print(f'adapted step_size per chain\n{fit.step_size}\nmetric_type: {fit.metric_type}\nmetric:\n{fit.metric}')

### Extracting the sample meta-data

In [ ]:
print('sample method variables:\n{}\n'.format(fit.metadata.method_vars.keys()))
print('stan model variables:\n{}'.format(fit.metadata.stan_vars.keys()))

## Saving the sampler output files

The sampler output files are written to a temporary directory which
is deleted upon session exit unless the ``output_dir`` argument is specified.
The ``save_csvfiles`` function moves the CmdStan CSV output files
to a specified directory without having to re-run the sampler.
The console output files are not saved. These files are treated as ephemeral; if the sample is valid, all relevant information is recorded in the CSV files.

## Parallelization via multi-threaded processing

Stan's multi-threaded processing is based on the Intel Threading Building Blocks (TBB) library, which must be linked to by the C++ compiler.   To take advantage of this option, you must compile (or recompile) the program with the the C++ compiler option `STAN_THREADS`.
The CmdStanModel object constructor and its `compile` method both have argument `cpp_options`
which takes as its value a dictionary of compiler flags.

We compile the example model `bernoulli.stan`, this time with arguments `cpp_options` and `compile`, and use the function `exe_info()` to check that the model has been compiled for multi-threading.

In [ ]:
model = CmdStanModel(stan_file='bernoulli.stan',
                     cpp_options={'STAN_THREADS': 'TRUE'},
                     compile='force')
model.exe_info()

### Cross-chain multi-threading

As of version CmdStan 2.28, it is possible to run the NUTS-HMC sampler on
multiple chains from within a single executable using threads.
This has the potential to speed up sampling.  It also
reduces the overall memory footprint required for sampling as
all chains share the same copy of data.the input data.
When using within-chain parallelization all chains started
within a single executable can share all the available threads
and once a chain finishes the threads will be reused.

The sample program argument ``parallel_chains`` takes an integer value which
specifies how many chains to run in parallel.
For models which have been compiled with option `STAN_THREADS` set, all chains are run from
within a single process and the value of the ``parallel_chains`` argument specifies the total number of threads.

In [ ]:
fit = model.sample(data='bernoulli.data.json', parallel_chains=4)

### Within-chain multi-threading

The Stan language
[reduce_sum](https://mc-stan.org/docs/stan-users-guide/reduce-sum.html)
function provides within-chain parallelization.
For models which require computing the sum of a number of independent function evaluations,
e.g., when evaluating a number of conditionally independent terms in a log-likelihood,
the `reduce_sum` function is used to parallelize this computation.

To see how this works, we run the "reflag" model, used in the 
[reduce_sum minimal example](https://mc-stan.org/users/documentation/case-studies/reduce_sum_tutorial.html) case study.
The Stan model and the original dataset are in files "redcard_reduce_sum.stan" and "redcard.json".

In [ ]:
with open('redcard_reduce_sum.stan', 'r') as fd:
    print(fd.read())

As before, we compile the model specifying argument `cpp_options`.

In [ ]:
redcard_model = CmdStanModel(stan_file='redcard_reduce_sum.stan',
                     cpp_options={'STAN_THREADS': 'TRUE'},
                     compile='force')
redcard_model.exe_info()

The `sample` method argument `threads_per_chain` specifies the number of threads allotted to each chain; this corresponds to CmdStan's `num_threads` argument.

In [ ]:
redcard_fit = redcard_model.sample(data='redcard.json', threads_per_chain=4)

The number of threads to use is passed to the model exe file by means of the shell environment variable `STAN_NUM_THREADS`.

On my machine, which has 4 cores, all 4 chains are run in parallel from within a single process.
Therefore, the total number of threads used by this process will be `threads_per_chain` * `chains`.
To check this, we examine the shell environment variable `STAN_NUM_THREADS`.

In [ ]:
os.environ['STAN_NUM_THREADS']